In [3]:
import numpy as np

In [4]:
def matrix_dot(A, B):
    if type(A[0]) != list:
        A = [A]

    if len(A[0]) != len(B):
        raise ValueError("행렬 곱 연산이 불가능합니다.")

    result = [[0 for _ in range(len(B[0]))] for _ in range(len(A))]
    for i in range(len(A)):
        for j in range(len(B[0])):
            dot_product = 0
            for k in range(len(A[0])):
                dot_product += A[i][k] * B[k][j]
            result[i][j] = dot_product
    return result

In [5]:
##########주어짐###########
#입력
X=[[i+0.1,i] for i in range(1000)] #한 시점에 특징이 2가지인 데이터 1000개
#print("X:",np.array(X).shape)
#print(X)
#배치 구현 <- simple_lstm 참고하여 바꿔야
BatchSize = int(input("배치 사이즈:"))
X=X[:BatchSize]
#print("X:",np.array(X).shape)
#print(X)
timeSize = int(input("한번에 고려할 시점 수:")) #한번에 고려할 시점의 개수
newX=[]
for i in range(len(X)-timeSize+1): # [0 1 2 3 4]를 2시점 씩 고려하면 [0 1] [1 2] [2 3] [3 4] 총 4가지가 되므로 
    smallX=[]
    for j in range(timeSize):
        smallX.append(X[i+j])
    newX.append(smallX)
X = newX
print("X:",np.array(X).shape)
print(X)
input_len = len(X)  #데이터의 개수만큼 target과 예측값이 있어야 하므로

#target
T=[[i//10] for i in range(input_len)] # 임의로 첫번째 값을 10으로 나눈 몫으로 정함 , 타겟이 마지막 층이라 원소가 1개인 target 값이 input_len만큼있음
print("T:",np.array(T).shape)
print(T)

#셔플수행하면 좋으나 그냥지나감

X: (96, 5, 2)
[[[0.1, 0], [1.1, 1], [2.1, 2], [3.1, 3], [4.1, 4]], [[1.1, 1], [2.1, 2], [3.1, 3], [4.1, 4], [5.1, 5]], [[2.1, 2], [3.1, 3], [4.1, 4], [5.1, 5], [6.1, 6]], [[3.1, 3], [4.1, 4], [5.1, 5], [6.1, 6], [7.1, 7]], [[4.1, 4], [5.1, 5], [6.1, 6], [7.1, 7], [8.1, 8]], [[5.1, 5], [6.1, 6], [7.1, 7], [8.1, 8], [9.1, 9]], [[6.1, 6], [7.1, 7], [8.1, 8], [9.1, 9], [10.1, 10]], [[7.1, 7], [8.1, 8], [9.1, 9], [10.1, 10], [11.1, 11]], [[8.1, 8], [9.1, 9], [10.1, 10], [11.1, 11], [12.1, 12]], [[9.1, 9], [10.1, 10], [11.1, 11], [12.1, 12], [13.1, 13]], [[10.1, 10], [11.1, 11], [12.1, 12], [13.1, 13], [14.1, 14]], [[11.1, 11], [12.1, 12], [13.1, 13], [14.1, 14], [15.1, 15]], [[12.1, 12], [13.1, 13], [14.1, 14], [15.1, 15], [16.1, 16]], [[13.1, 13], [14.1, 14], [15.1, 15], [16.1, 16], [17.1, 17]], [[14.1, 14], [15.1, 15], [16.1, 16], [17.1, 17], [18.1, 18]], [[15.1, 15], [16.1, 16], [17.1, 17], [18.1, 18], [19.1, 19]], [[16.1, 16], [17.1, 17], [18.1, 18], [19.1, 19], [20.1, 20]], [[17.1, 17]

In [6]:
outputQ = input("LSTM 한 층의 입력하나요(Y/N)?:") #층이 하나면 그냥 target의 종류, 층이 여러개면 정해줘야 함. 현재는 단일 모듈이므로 target의 종류로 정함
count_list=[] # 타겟의 종류 수 카운트 => layer output의 크기를 정하기 위해서(위의 ? 정함) lstm 층이 여러개면 lstm 모듈마다 output_size 정해야
cnt = 0
for now in T:
    if now in count_list:
        pass
    else:
        cnt += 1
        count_list.append(now)
output_size = cnt #w의 output 크기
if(outputQ == "N"):
    output_size = int(input("output_size:")) 
print(output_size)

10


In [7]:
##########작성###########
#state초기화
Y = [[[0 for i in range(output_size)] for i in range(timeSize)] for _ in range(input_len)] # 실제 값은 X의 입력 만큼 있어야 하므로
C = [[[0 for i in range(timeSize)] for i in range(timeSize)] for _ in range(input_len)] # X은 시점 수 만큼 있어야 하므로
print("Y:",np.array(Y).shape)
print(Y)
print("C:",np.array(C).shape)
print(C)

Y: (96, 5, 10)
[[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], [[0, 0, 0, 0

In [8]:
#parameter초기화
input_feature = len(X[0][0]) # 한 시점에서 데이터의 특징 수 <= X와 w의 행렬곱을 수행해야하므로 X가 (시점수,2)이므로 w는 (2,?)가 되야함
print(input_feature)

2


In [9]:
#parameter 초기화
import random
#가중치 초기화: 가우시안 초기화 방법,  He 초기화 (He Initialization) 또는 Xavier 초기화 (Glorot Initialization)도 존재
w_matrix = []
v_matrix = []
b_matrix = []
for _ in range(4): ## 망각 게이트,입력 게이트,새로운 기억, 출력 게이트 총 4개에서 업데이트 해야함
    w0=[[random.gauss(0,1) for i in range(output_size)] for j in range(input_feature)] # X는 (시점 수, 특징 수)이므로 w 는 (특징수, output_size) 
    #print(np.array(w0).shape)
    v0=[[random.gauss(0,1) for i in range(output_size)]for j in range(output_size)]  # Y는 (시점 수, 특징 수)이므로 w 는 (특징수, output_size)
    #print(np.array(v0).shape)
    b0=[[random.gauss(0,1) for i in range(output_size)]for j in range(timeSize)] 
    #print(np.array(b0).shape)
    w_matrix.append(w0)
    v_matrix.append(v0)
    b_matrix.append(b0)
print('X')
print(np.array(X).shape)
print('w_matrix')
print(np.array(w_matrix).shape)
print('Y')
print(np.array(Y).shape)
print('v_matrix')
print(np.array(v_matrix).shape)
################################
print('b_matrix')
print(np.array(b_matrix).shape)


X
(96, 5, 2)
w_matrix
(4, 2, 10)
Y
(96, 5, 10)
v_matrix
(4, 10, 10)
b_matrix
(4, 5, 10)


In [10]:
now = 1
nowtime = 1
print("========  matrix multiplication (Xw)  ========\n")
#print(np.matmul(X[now],w_matrix[0]).shape) .matnul는 2차원만 연산 가능
#print(np.matmul(X[now],w_matrix[0]))
print("-----------------Using Numpy------------------")
print(np.dot(X[now],w_matrix[0]).shape)
print(np.dot(X[now],w_matrix[0]))
print("\n----------------What I made------------------")
print(np.array(matrix_dot(X[nowtime],w_matrix[0])).shape)
print(np.array(matrix_dot(X[nowtime],w_matrix[0])))


========  matrix multiplication (Xw)  ========

-----------------Using Numpy------------------
(5, 10)
[[ 1.67229381 -1.45267024 -1.88659246  1.00249059 -0.05887344 -0.30635941
   1.96480091  2.18821812  0.42164935  2.1891646 ]
 [ 3.24983683 -2.90544421 -3.6717334   1.98871962 -0.06829382 -0.62556396
   3.83991768  4.35793096  0.86786889  4.28531007]
 [ 4.82737986 -4.35821818 -5.45687434  2.97494864 -0.0777142  -0.94476851
   5.71503445  6.52764379  1.31408842  6.38145555]
 [ 6.40492288 -5.81099215 -7.24201529  3.96117766 -0.08713459 -1.26397306
   7.59015122  8.69735663  1.76030796  8.47760103]
 [ 7.98246591 -7.26376612 -9.02715623  4.94740668 -0.09655497 -1.58317761
   9.46526798 10.86706947  2.2065275  10.57374651]]

----------------What I made------------------
(5, 10)
[[ 1.67229381 -1.45267024 -1.88659246  1.00249059 -0.05887344 -0.30635941
   1.96480091  2.18821812  0.42164935  2.1891646 ]
 [ 3.24983683 -2.90544421 -3.6717334   1.98871962 -0.06829382 -0.62556396
   3.83991768  4.

In [11]:
print("==================")
print("Yv")
print(np.dot(Y[now],v_matrix[0]).shape)
print(np.dot(Y[now],v_matrix[0]))
print(np.array(matrix_dot(Y[nowtime],v_matrix[0])).shape)
print(np.array(matrix_dot(Y[nowtime],v_matrix[0])))

Yv
(5, 10)
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
(5, 10)
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]


In [12]:
def element_sum(arr1, arr2, arr3):
    if len(arr1) != len(arr2) or len(arr1) != len(arr3) or len(arr1[0]) != len(arr2[0]) or len(arr1[0]) != len(arr3[0]):
        raise ValueError("Input arrays must have the same shape")
    #print(np.array(arr1))
    #print(np.array(arr2))
    #print(np.array(arr3))
    result = []
    for i in range(len(arr1)):
        row = []
        for j in range(len(arr1[0])):
            row.append(arr1[i][j] + arr2[i][j] + arr3[i][j])
        result.append(row)
    
    return result

In [13]:
print("========   elemental sum  (Xw+Yv+b) ==========\n")
print("-----------------Using Numpy------------------")
print((np.dot(X[now],w_matrix[0])+np.dot(Y[now],v_matrix[0])+b_matrix[0]).shape)
print((np.dot(X[now],w_matrix[0])+np.dot(Y[now],v_matrix[0])+b_matrix[0]))
print("\n----------------What I made------------------")
Xw = matrix_dot(X[nowtime],w_matrix[0])
Yv = matrix_dot(Y[nowtime],v_matrix[0])
Xw_Yv_b = element_sum(Xw,Yv,b_matrix[0])
print(np.array(Xw_Yv_b).shape)
print(np.array(Xw_Yv_b))


========   elemental sum  (Xw+Yv+b) ==========

-----------------Using Numpy------------------
(5, 10)
[[ 3.33647416e+00 -8.92246917e-01 -1.24360967e+00  3.27185794e+00
   2.23541056e+00 -3.58746568e-03  2.93777327e-01  2.41909889e+00
   1.24779241e+00  2.26640826e+00]
 [ 2.21425486e+00 -4.15817682e+00 -2.79092602e+00  2.71837723e+00
  -4.42159402e-01  9.13375793e-01  2.70795033e+00  5.70473756e+00
   7.75270329e-01  4.28280829e+00]
 [ 5.34313038e+00 -4.73798144e+00 -5.42733622e+00  4.17756794e+00
   1.48005196e+00 -1.71905744e+00  3.48706650e+00  7.90620672e+00
   1.41448127e+00  5.96599819e+00]
 [ 6.00013770e+00 -4.16309545e+00 -8.43177017e+00  2.55284079e+00
  -2.77183337e-01 -2.98997360e+00  8.14538834e+00  9.02942161e+00
   6.70403389e-01  7.85178383e+00]
 [ 9.46584114e+00 -8.04761089e+00 -1.01291617e+01  5.93613712e+00
   1.56229364e+00 -2.30470292e+00  8.67064947e+00  1.04466920e+01
   3.00579846e+00  1.06887190e+01]]

----------------What I made------------------
(5, 10)
[[ 3.3

In [24]:
def activation(function,x):
    e = 2.718281828459045 #자연상수:2.71828182845904523536
    result = []
    if(function == 'Sigmoid'):
        for i in range(len(x)):
            row = []
            for j in range(len(x[0])):
                now_x = x[i][j]
                row.append((e**now_x)/((e**now_x)+1)) #C언어에서는 **이 없어 **도 구현
            result.append(row)
        return result
    if(function == 'Tanh'):
        for i in range(len(x)):
            row = []
            for j in range(len(x[0])):
                now_x = x[i][j]
                row.append((2/(1+(e**(-2*now_x))))-1)
            result.append(row)
        return result
    if(function == 'ReLU'):
        if (x<=0):
            return 0
        return x
    return [result]

In [25]:
print("======== Activate Fuction (Sigmoid) ==========\n")
print("-----------------Using Numpy------------------")
def sigmoid(x):
    return 1 / (1 + np.exp(-x))
print(sigmoid(np.array(Xw_Yv_b)))
print("\n----------------What I made------------------")
print(np.array(activation('Sigmoid',Xw_Yv_b)))

======== Activate Fuction (Sigmoid) ==========

-----------------Using Numpy------------------
[[2.25587066e-01 4.52865867e-02 3.82893547e-01 9.38106401e-01
  9.32009266e-01 3.52428306e-01 8.28652837e-01 3.64139338e-01
  6.28921055e-02 2.60676868e-01]
 [1.92444283e-01 1.96449474e-02 7.02880109e-01 6.64659953e-01
  9.69477868e-01 4.58120124e-01 9.86157327e-01 6.57136151e-01
  1.92592024e-03 2.26693044e-01]
 [1.89797146e-01 7.62283310e-03 7.85688481e-01 9.46133473e-01
  9.99497623e-01 1.07107688e-01 9.95257152e-01 9.11936030e-01
  2.72690891e-05 1.99543079e-01]
 [3.60842068e-01 2.74862032e-02 2.28085095e-01 9.92390584e-01
  9.99527537e-01 4.99598019e-02 9.98919165e-01 9.54626285e-01
  4.73691334e-07 2.04121710e-01]
 [1.32813927e-01 1.68833618e-03 8.68279000e-01 9.98728511e-01
  9.99824854e-01 3.45501882e-02 9.99895396e-01 9.47406397e-01
  1.83446802e-08 1.82885068e-02]]

----------------What I made------------------
[[2.25587066e-01 4.52865867e-02 3.82893547e-01 9.38106401e-01
  9.320092

In [26]:
print("======== Activate Fuction (Tanh) ==========\n")
print("-----------------Using Numpy------------------")
npX=np.tanh(Xw_Yv_b)
print(npX)
print("\n----------------What I made------------------")
print(np.array(activation('Tanh',Xw_Yv_b)))

======== Activate Fuction (Tanh) ==========

-----------------Using Numpy------------------
[[-0.84356244 -0.99550999 -0.44406625  0.99133176  0.98941273 -0.54298736
   0.91799215 -0.50607768 -0.99103209 -0.77885508]
 [-0.89252519 -0.99919723  0.69679857  0.59419808  0.9980196  -0.16635243
   0.999606    0.57204535 -0.99999255 -0.84173003]
 [-0.89595531 -0.999882    0.86149895  0.99353814  0.99999949 -0.97162941
   0.99995458  0.98152151 -1.         -0.88298431]
 [-0.51661488 -0.99840368 -0.8394049   0.99988242  0.99999955 -0.99448446
   0.99999766  0.99549192 -1.         -0.87656223]
 [-0.95416225 -0.99999428  0.95500753  0.99999676  0.99999994 -0.99744191
   0.99999998  0.9938555  -1.         -0.99930615]]

----------------What I made------------------
(5, 10)
[[-0.84356244 -0.99550999 -0.44406625  0.99133176  0.98941273 -0.54298736
   0.91799215 -0.50607768 -0.99103209 -0.77885508]
 [-0.89252519 -0.99919723  0.69679857  0.59419808  0.9980196  -0.16635243
   0.999606    0.57204535 -0

In [17]:
###################################LSTM###################################
nowtime = 1
pasttime = nowtime-1
#forgot gate
print("===========forgot gate============")
Xw = matrix_dot(X[nowtime],w_matrix[0])
Yv = matrix_dot(Y[pasttime],v_matrix[0])
Xw_Yv_b = element_sum(Xw,Yv,b_matrix[0])
fGate = activation('Sigmoid',Xw_Yv_b)
print(np.array(fGate).shape)
print(np.array(fGate))

===========forgot gate============
(5, 10)
[[9.65659111e-01 2.90646360e-01 2.23808295e-01 9.63450653e-01
  9.03384629e-01 4.99103135e-01 5.72920633e-01 9.18272143e-01
  7.76917482e-01 9.06056511e-01]
 [9.01522318e-01 1.53953175e-02 5.78164909e-02 9.38102373e-01
  3.91226547e-01 7.13690457e-01 9.37494148e-01 9.96680902e-01
  6.84659867e-01 9.86384109e-01]
 [9.95241867e-01 8.68029614e-03 4.37555733e-03 9.84895874e-01
  8.14580429e-01 1.51992611e-01 9.70317523e-01 9.99631686e-01
  8.04471796e-01 9.97442076e-01]
 [9.97527716e-01 1.53209368e-02 2.17788100e-04 9.27764130e-01
  4.31144453e-01 4.78808934e-02 9.99710014e-01 9.99880183e-01
  6.61593479e-01 9.99611094e-01]
 [9.99922553e-01 3.19762923e-04 3.98973130e-05 9.97364746e-01
  8.26682230e-01 9.07342152e-02 9.99828482e-01 9.99970957e-01
  9.52835396e-01 9.99977200e-01]]


In [18]:
#input gate
print("===========input gate============")
Xw = matrix_dot(X[nowtime],w_matrix[1])
Yv = matrix_dot(Y[pasttime],v_matrix[1])
Xw_Yv_b = element_sum(Xw,Yv,b_matrix[1])
iGate = activation('Sigmoid',Xw_Yv_b)
print(np.array(iGate).shape)
print(np.array(iGate))

===========input gate============
(5, 10)
[[8.98331996e-01 1.68160644e-01 6.43062799e-01 2.40242304e-01
  9.26219538e-01 9.82130017e-01 1.62984214e-01 1.50340292e-01
  4.66590276e-01 6.80102892e-02]
 [9.97454670e-01 2.95900770e-01 6.35285973e-01 7.81671795e-01
  9.84614131e-01 9.90890496e-01 3.61927461e-01 2.58924076e-01
  3.59689539e-01 3.94784474e-02]
 [9.99868846e-01 8.70078201e-01 4.91284940e-01 6.61114901e-01
  9.91769463e-01 9.99933047e-01 7.52277243e-01 7.95853202e-02
  2.11568660e-01 5.31282105e-02]
 [9.99899382e-01 1.23726689e-01 3.35300251e-01 3.72599979e-01
  9.93924560e-01 9.99993644e-01 1.79183940e-01 4.79971989e-02
  6.78960129e-01 1.11619725e-02]
 [9.99999420e-01 1.72999484e-01 4.64107535e-01 6.83966219e-01
  9.96462342e-01 9.99999163e-01 6.05812691e-01 2.19329370e-02
  2.67868158e-01 5.72382632e-04]]


In [19]:
#new input
print("===========new input============")
Xw = matrix_dot(X[nowtime],w_matrix[2])
Yv = matrix_dot(Y[pasttime],v_matrix[2])
Xw_Yv_b = element_sum(Xw,Yv,b_matrix[2])
newIn = activation('Tanh',Xw_Yv_b)
print(np.array(newIn).shape)
print(np.array(newIn))

===========new input============
(5, 10)
(5, 10)
[[-0.84356244 -0.99550999 -0.44406625  0.99133176  0.98941273 -0.54298736
   0.91799215 -0.50607768 -0.99103209 -0.77885508]
 [-0.89252519 -0.99919723  0.69679857  0.59419808  0.9980196  -0.16635243
   0.999606    0.57204535 -0.99999255 -0.84173003]
 [-0.89595531 -0.999882    0.86149895  0.99353814  0.99999949 -0.97162941
   0.99995458  0.98152151 -1.         -0.88298431]
 [-0.51661488 -0.99840368 -0.8394049   0.99988242  0.99999955 -0.99448446
   0.99999766  0.99549192 -1.         -0.87656223]
 [-0.95416225 -0.99999428  0.95500753  0.99999676  0.99999994 -0.99744191
   0.99999998  0.9938555  -1.         -0.99930615]]
(5, 10)
[[-0.84356244 -0.99550999 -0.44406625  0.99133176  0.98941273 -0.54298736
   0.91799215 -0.50607768 -0.99103209 -0.77885508]
 [-0.89252519 -0.99919723  0.69679857  0.59419808  0.9980196  -0.16635243
   0.999606    0.57204535 -0.99999255 -0.84173003]
 [-0.89595531 -0.999882    0.86149895  0.99353814  0.99999949 -0.97

In [21]:
#output gate
print("===========output gate============")
Xw = matrix_dot(X[nowtime],w_matrix[3])
Yv = matrix_dot(Y[pasttime],v_matrix[3])
Xw_Yv_b = element_sum(Xw,Yv,b_matrix[3])
oGate = activation('Sigmoid',Xw_Yv_b)
print(np.array(oGate).shape)
print(np.array(oGate))

===========output gate============
(5, 10)
[[8.51164433e-01 3.76406771e-01 7.34284967e-01 3.27483034e-01
  6.00546909e-01 1.22349247e-02 7.34386200e-01 7.64461662e-01
  3.94840372e-02 4.87373747e-01]
 [7.80920064e-01 9.22446928e-01 7.29444483e-01 9.79395628e-01
  9.40614401e-01 3.03940107e-02 9.32916280e-01 2.76315956e-01
  3.37131373e-02 3.20736346e-01]
 [9.75883622e-01 9.77282531e-01 9.58773280e-01 9.80851046e-01
  4.77254791e-01 1.15422332e-02 9.67151794e-01 6.00883703e-01
  1.43820848e-02 8.02036752e-01]
 [9.53404929e-01 9.94725346e-01 6.36129747e-01 9.94757483e-01
  8.69957762e-01 3.78365182e-04 9.43570174e-01 4.55958724e-01
  1.45472488e-02 5.03551844e-01]
 [9.75136752e-01 9.97998687e-01 9.89349224e-01 9.99655386e-01
  8.34456579e-01 1.01386536e-04 7.74126328e-01 7.42361523e-01
  2.09238201e-03 4.84048801e-01]]


In [120]:
"""
#망각gate
nowtime = 1
fCalculate = element_sum(X[nowtime],w_matrix[0],Y[nowtime],v_matrix[0],b_matrix[0])
fGate = matrix_dot(C[nowtime], activation("Sigmoid",fCalculate[0])) # X는 시점 t, Y는 시점 t-1
C[nowtime] = D1vectorsum(fGate[0])
"""
#print(C)
print(np.array(activation("Sigmoid",fCalculate[0])).shape)
#입력gate
iCalculate = element_sum(X[nowtime],w_matrix[1],Y[nowtime],v_matrix[1],b_matrix[1])
nCalculate = element_sum(X[nowtime],w_matrix[2],Y[nowtime],v_matrix[2],b_matrix[2])
print(np.array(activation("Sigmoid",iCalculate[0])).shape) # 
print(np.array(activation("Tanh",nCalculate[0])).shape)
#iGate = matrix_dot(activation("Sigmoid",iCalculate[0]),activation("Tanh",nCalculate[0]))

iGate = C +(activation("Sigmoid",iCalaulate[0]) * activation("Tanh",nCalaulate[0]))
C = iGate
#출력gate
oCalculate = element_sum(X[nowtime],w_matrix[3],Y[nowtime],v_matrix[3],b_matrix[3])
oGate = activation("Sigmoid",oCalculate) * activation("Tanh",C) 
Y = oGate
"""

(1, 10)
(1, 10)
(1, 10)


'\niGate = C +(activation("Sigmoid",iCalaulate[0]) * activation("Tanh",nCalaulate[0]))\nC = iGate\n#출력gate\noCalculate = element_sum(X[nowtime],w_matrix[3],Y[nowtime],v_matrix[3],b_matrix[3])\noGate = activation("Sigmoid",oCalculate) * activation("Tanh",C) \nY = oGate\n'

In [68]:
print(C)

[[0], [0.0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0]]


In [44]:
print(C[1])

[0.0]


In [48]:
C[1]= [2]

In [49]:
print(C)

[[0.0], [2], [0.0], [0.0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0]]
